# Clasificación de sonido ambiente 
---
# Preparación de datos

## Extraer características
Posteriormente extraemos las características propias de cada imagen que nos van a permitir entrenar el modelo.

<p>Para ello vamos a crear una representación visual de cada una de las muestras de audio que nos permitirá identificar características para la clasificación, utilizando las mismas técnicas utilizadas para clasificar imágenes. 
    
<p>Los <strong>espectrogramas</strong> son una técnica útil para visualizar el espectro de frecuencias de un sonido y cómo varían durante un período de tiempo muy corto. Utilizaremos una técnica similar conocida como <a href="https://en.wikipedia.org/wiki/Mel-frequency_cepstrum" target="_blank" rel="noreferrer noopener">Mel-Frequency Cepstral Coefficients (MFCC)</a>.</p>

<p>La principal diferencia es que un espectrograma usa una <strong>escala de frecuencia lineal espaciada</strong> (por lo que cada intervalo de frecuencia está espaciado un número igual de Hertz), mientras que un MFCC <strong>usa una escala de frecuencia espaciada cuasi-logarítmica</strong>, que es más similar a cómo el sistema auditivo humano procesa sonidos. </p>

<p>La imagen a continuación compara tres representaciones visuales diferentes de una onda de sonido, la primera es la representación en el dominio del tiempo, comparando la amplitud en el tiempo. El siguiente es un espectrograma que muestra la energía en diferentes bandas de frecuencia que cambian con el tiempo, y finalmente un MFCC que podemos ver es muy similar a un espectrograma pero con detalles más distinguibles.</p>
<img src="https://www.diegocalvo.es/wp-content/uploads/2020/06/image-31-1024x557.png">


Para cada archivo de audio en el conjunto de datos, extraeremos un MFCC (lo que significa que tenemos una representación de imagen para cada muestra de audio) y lo almacenaremos en un Panda Dataframe junto con su etiqueta de clasificación. Para esto, utilizaremos la función mfcc() de Librosa, que genera un MFCC a partir de datos de audio de series temporales.

In [187]:
# Set the path to the full UrbanSound dataset 
fulldatasetpath = 'audio/'

# Set the path to metadata.
metadata = pd.read_csv('audio/UrbanSound8K.csv')

In [168]:
import numpy as np

def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccsscaled

In [169]:
# Load various imports 
import pandas as pd
import os
import librosa

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
    class_label = row["classID"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files') 

Finished feature extraction from  40  files


In [170]:

features = featuresdf.loc[1]
print( list(features) )

[array([-4.59564667e+02,  1.22800354e+02, -4.79247093e+01,  5.32656937e+01,
       -1.13986862e+00,  2.47723713e+01,  7.98013592e+00,  9.67425823e+00,
       -1.02354348e+00,  1.12044487e+01, -1.18449421e+01,  1.43469524e+01,
       -3.51148415e+00,  1.14010563e+01, -3.67504358e-01,  6.43268776e+00,
       -6.60325956e+00,  3.28451920e+00, -2.08006835e+00,  1.09049597e+01,
        1.38384926e+00,  6.04806185e+00, -8.37760806e-01,  1.80902553e+00,
       -2.65425372e+00,  1.58228743e+00, -1.59489763e+00, -6.00140512e-01,
       -1.92130005e+00,  1.28997445e+00,  1.24426317e+00, -1.45356560e+00,
       -3.94021797e+00, -2.18472099e+00, -1.45620716e+00, -1.69737124e+00,
       -1.49213314e+00,  2.22541404e+00,  1.54661775e+00, -8.36315870e-01],
      dtype=float32), 1]


## Convertir los datos y etiquetas
Para transformarlos datos categóricos a numéricos usaremos "LabelEncoder" y así conseguiremos que el modelo sea capaz de entenderlos.

In [171]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

In [174]:
print(X)

[[-4.24686768e+02  1.10562271e+02 -5.41482353e+01 ...  6.17408633e-01
  -6.84974134e-01  5.71514487e-01]
 [-4.59564667e+02  1.22800354e+02 -4.79247093e+01 ...  2.22541404e+00
   1.54661775e+00 -8.36315870e-01]
 [-4.14553772e+02  1.02896904e+02 -3.66649513e+01 ... -1.24488878e+00
  -2.17944264e+00 -1.07717764e+00]
 ...
 [-3.17793388e+01  1.55154846e+02 -4.89359894e+01 ... -6.00718446e-02
  -3.32850844e-01  3.64527069e-02]
 [-6.25979347e+01  1.88543030e+02 -4.80912743e+01 ... -4.71712708e-01
  -3.67541552e-01 -6.41354263e-01]
 [-5.66862221e+01  1.71027161e+02 -5.16182098e+01 ... -2.69095808e-01
  -8.75584364e-01 -2.98762083e-01]]


In [176]:
print(y)

[ 1  1  1  1  2  2  2  2  3  3  3  3  4  4  4  4  5  5  5  5  6  6  6  6
  7  7  7  7  8  8  8  8  9  9  9  9 10 10 10 10]


In [173]:
print(yy)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0.

## Dividir los datos en entrenamiento y test
Dividimos el conjunto de datos en dos bloques (80% y 20%) y de ellos sacamos valores de X y de Y

In [178]:
# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [179]:
x_train.shape

(32, 40)

In [180]:
x_test.shape

(8, 40)

In [188]:
### store the preprocessed data for use in the next notebook

%store x_train 
%store x_test 
%store y_train 
%store y_test 
%store yy 
%store le
%store metadata
%store fulldatasetpath

Stored 'x_train' (ndarray)
Stored 'x_test' (ndarray)
Stored 'y_train' (ndarray)
Stored 'y_test' (ndarray)
Stored 'yy' (ndarray)
Stored 'le' (LabelEncoder)
Stored 'metadata' (DataFrame)
Stored 'fulldatasetpath' (str)
